<a href="https://colab.research.google.com/github/krsna-thecoder/CNN---DEEP-LEARNING/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install pennylane tensorflow numpy matplotlib pillow scikit-learn


In [21]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from PIL import Image
from sklearn.decomposition import PCA



In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Select only classes 0 and 1
train_filter = np.where((y_train == 0) | (y_train == 1))
test_filter = np.where((y_test == 0) | (y_test == 1))

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [24]:
from sklearn.decomposition import PCA

# ---- PCA FEATURE EXTRACTION (4 components to match 2x2 input and n_qubits) ----

# Flatten images: (N, 28, 28) → (N, 784) - Note: This comment might be outdated if images are already resized to 2x2.
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Normalize ONCE (before PCA) - Note: This might be redundant if data is already normalized elsewhere.
x_train_flat = x_train_flat / 255.0
x_test_flat = x_test_flat / 255.0

# Apply PCA
pca = PCA(n_components=4) # Changed n_components from 6 to 4
x_train = pca.fit_transform(x_train_flat)
x_test = pca.transform(x_test_flat)

In [25]:
n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)


In [6]:
def encode(x):
    for i in range(n_qubits):
        qml.RY(np.pi * x[i], wires=i)


In [26]:
def quantum_conv(weights):
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)

    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])


In [27]:
def pooling():
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[3, 2])
    qml.CNOT(wires=[5, 4])


In [28]:
@qml.qnode(dev)
def qcnn(weights, x):
    encode(x)
    quantum_conv(weights)
    pooling()
    return qml.expval(qml.PauliZ(0))


In [29]:
def predict(weights, x):
    return qcnn(weights, x)

def loss(weights, X, Y):
    loss_val = 0
    for x, y in zip(X, Y):
        pred = predict(weights, x)
        loss_val += (pred - (1 - 2*y))**2
    return loss_val / len(X)


In [31]:
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from PIL import Image

# Data loading and preprocessing
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Select only classes 0 and 1
train_filter = np.where((y_train == 0) | (y_train == 1))
test_filter = np.where((y_test == 0) | (y_test == 1))

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

def resize_images(images):
    resized = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((2, 2))
        resized.append(np.array(img).flatten())
    return np.array(resized)

x_train = resize_images(x_train)
x_test = resize_images(x_test)

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

def encode(x):
    for i in range(n_qubits):
        qml.RY(np.pi * x[i], wires=i)

def quantum_conv(weights):
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

def pooling():
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[3, 2])

@qml.qnode(dev)
def qcnn(weights, x):
    encode(x)
    quantum_conv(weights)
    pooling()
    return qml.expval(qml.PauliZ(0))

def predict(weights, x):
    return qcnn(weights, x)

def loss(weights, X, Y):
    loss_val = 0
    for x, y in zip(X, Y):
        pred = predict(weights, x)
        loss_val += (pred - (1 - 2*y))**2
    return loss_val / len(X)

np.random.seed(0)
weights = np.random.randn(n_qubits, requires_grad=True)

optimizer = qml.AdamOptimizer(stepsize=0.1)
epochs = 20

for epoch in range(epochs):
    weights = optimizer.step(lambda w: loss(w, x_train[:50], y_train[:50]), weights)
    train_loss = loss(weights, x_train[:50], y_train[:50])
    print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}")

/tmp/ipython-input-1553767310.py:64: RuntimeWarning: overflow encountered in scalar subtract
  loss_val += (pred - (1 - 2*y))**2


Epoch 1, Loss: 33750.5816
Epoch 2, Loss: 33725.5431
Epoch 3, Loss: 33701.4289
Epoch 4, Loss: 33678.4994
Epoch 5, Loss: 33657.0041
Epoch 6, Loss: 33637.1747
Epoch 7, Loss: 33619.2173
Epoch 8, Loss: 33603.3038
Epoch 9, Loss: 33589.5629
Epoch 10, Loss: 33578.0707
Epoch 11, Loss: 33568.8421
Epoch 12, Loss: 33561.8247
Epoch 13, Loss: 33556.8955
Epoch 14, Loss: 33553.8625
Epoch 15, Loss: 33552.4721
Epoch 16, Loss: 33552.4224
Epoch 17, Loss: 33553.3824
Epoch 18, Loss: 33555.0139
Epoch 19, Loss: 33556.9945
Epoch 20, Loss: 33559.0380


In [50]:
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from PIL import Image

# Data loading and preprocessing
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Select only classes 0 and 1
train_filter = np.where((y_train == 0) | (y_train == 1))
test_filter = np.where((y_test == 0) | (y_test == 1))

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

def resize_images(images):
    resized = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((2, 2))
        resized.append(np.array(img).flatten())
    return np.array(resized)

x_train = resize_images(x_train)
x_test = resize_images(x_test)

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

def encode(x):
    for i in range(n_qubits):
        qml.RY(np.pi * x[i], wires=i)

def quantum_conv(weights):
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

def pooling():
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[3, 2])

@qml.qnode(dev)
def qcnn(weights, x):
    encode(x)
    quantum_conv(weights)
    pooling()
    return qml.expval(qml.PauliZ(0))

def predict(weights, x):
    return qcnn(weights, x)

def loss(weights, X, Y):
    loss_val = 0
    for x, y in zip(X, Y):
        pred = qcnn(weights, x)          # ⟨Z⟩ in [-1, 1]
        prob = (1 + pred) / 2            # map to [0, 1]
        loss_val += -(y * np.log(prob + 1e-8) +
                      (1 - y) * np.log(1 - prob + 1e-8))
    return loss_val / len(X)

np.random.seed(0)
weights = np.random.randn(n_qubits, requires_grad=True)

optimizer = qml.AdamOptimizer(stepsize=0.1)
epochs = 40

for epoch in range(epochs):
    weights = optimizer.step(lambda w: loss(w, x_train[:50], y_train[:50]), weights)
    train_loss = loss(weights, x_train[:200], y_train[:200])
    print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}")

Epoch 1, Loss: 0.6248
Epoch 2, Loss: 0.6244
Epoch 3, Loss: 0.6268
Epoch 4, Loss: 0.6287
Epoch 5, Loss: 0.6277
Epoch 6, Loss: 0.6258
Epoch 7, Loss: 0.6246
Epoch 8, Loss: 0.6243
Epoch 9, Loss: 0.6243
Epoch 10, Loss: 0.6245
Epoch 11, Loss: 0.6252
Epoch 12, Loss: 0.6262
Epoch 13, Loss: 0.6269
Epoch 14, Loss: 0.6267
Epoch 15, Loss: 0.6259
Epoch 16, Loss: 0.6251
Epoch 17, Loss: 0.6247
Epoch 18, Loss: 0.6245
Epoch 19, Loss: 0.6246
Epoch 20, Loss: 0.6248
Epoch 21, Loss: 0.6253
Epoch 22, Loss: 0.6259
Epoch 23, Loss: 0.6261
Epoch 24, Loss: 0.6260
Epoch 25, Loss: 0.6255
Epoch 26, Loss: 0.6250
Epoch 27, Loss: 0.6248
Epoch 28, Loss: 0.6247
Epoch 29, Loss: 0.6248
Epoch 30, Loss: 0.6251
Epoch 31, Loss: 0.6255
Epoch 32, Loss: 0.6257
Epoch 33, Loss: 0.6257
Epoch 34, Loss: 0.6255
Epoch 35, Loss: 0.6252
Epoch 36, Loss: 0.6249
Epoch 37, Loss: 0.6249
Epoch 38, Loss: 0.6250
Epoch 39, Loss: 0.6252
Epoch 40, Loss: 0.6254


In [54]:
def accuracy(weights, X, Y):
    correct = 0
    for x, y in zip(X, Y):
        pred = predict(weights, x)
        label = 0 if pred > 0 else 1
        if label == y:
            correct += 1
    return correct / len(X)

print("Test accuracy:", accuracy(weights, x_test[:30], y_test[:30]))


Test accuracy: 0.3333333333333333


In [53]:
preds = [predict(weights, x) for x in x_test[:20]]
print(preds)

[tensor(-0.04754734, requires_grad=True), tensor(0.37429252, requires_grad=True), tensor(-0.03523801, requires_grad=True), tensor(0.30478615, requires_grad=True), tensor(-0.43867556, requires_grad=True), tensor(0.35133233, requires_grad=True), tensor(-0.2660806, requires_grad=True), tensor(0.39702547, requires_grad=True), tensor(-0.13340616, requires_grad=True), tensor(0.07556234, requires_grad=True), tensor(0.39702547, requires_grad=True), tensor(-0.08442458, requires_grad=True), tensor(0.52788918, requires_grad=True), tensor(0.4854112, requires_grad=True), tensor(-0.03523801, requires_grad=True), tensor(0.57917244, requires_grad=True), tensor(-0.46068273, requires_grad=True), tensor(-0.42757126, requires_grad=True), tensor(0.18543466, requires_grad=True), tensor(0.50680403, requires_grad=True)]


In [55]:
def predict_mnist_image(img, weights):
    # img: (28, 28) MNIST image

    img_flat = img.reshape(1, -1) / 255.0
    img_pca = pca.transform(img_flat)  # SAME pca object

    pred = qcnn(weights, img_pca[0])
    return "T-shirt/top" if pred > 0 else "Trouser"


In [56]:
print(predict_image("/content/t shirt for testing of qcnn.png", weights))


T-shirt/top


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [57]:
print(predict_image("/content/crop top image.png", weights))


Trouser


In [58]:
print(predict_image("/content/trousers for test.png", weights))


T-shirt/top


In [59]:
print(predict_image("/content/trouser 3 for test.png", weights))


Trouser


In [60]:
print(predict_image("/content/trouser 2 for test.png", weights))


T-shirt/top


In [61]:
print(predict_image("/content/trouser 4 for test.png", weights))


Trouser


In [62]:
preds = [predict(weights, x) for x in x_test[:20]]
print(preds)

[tensor(-0.04754734, requires_grad=True), tensor(0.37429252, requires_grad=True), tensor(-0.03523801, requires_grad=True), tensor(0.30478615, requires_grad=True), tensor(-0.43867556, requires_grad=True), tensor(0.35133233, requires_grad=True), tensor(-0.2660806, requires_grad=True), tensor(0.39702547, requires_grad=True), tensor(-0.13340616, requires_grad=True), tensor(0.07556234, requires_grad=True), tensor(0.39702547, requires_grad=True), tensor(-0.08442458, requires_grad=True), tensor(0.52788918, requires_grad=True), tensor(0.4854112, requires_grad=True), tensor(-0.03523801, requires_grad=True), tensor(0.57917244, requires_grad=True), tensor(-0.46068273, requires_grad=True), tensor(-0.42757126, requires_grad=True), tensor(0.18543466, requires_grad=True), tensor(0.50680403, requires_grad=True)]
